In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,time,pickle
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from joblib import dump, load
from utils import data_process, visualize
from utils.utils import train, test, summary, setup_seed
from utils.TFEQ import TFEQ,TFEQ_Conv,TFEQ_RNN

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [2]:
data = np.load("./data_4/data_train.npy", allow_pickle=True)
x_train = data.item().get("x_train")
y_train = data.item().get("y_train")
data = np.load("./data_4/data_test.npy", allow_pickle=True)
x_test = data.item().get("x_test")
y_test = data.item().get("y_test")
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()
traindata = TensorDataset(x_train, y_train)
testdata = TensorDataset(x_test, y_test)

train_loader = DataLoader(traindata, batch_size=128, shuffle=True)
test_loader = DataLoader(testdata, batch_size=256, shuffle=True)

In [9]:
%%time
setup_seed(42)
# model = CRNN().cuda()
model=TFEQ(channel=3, time_in=400).cuda()
L = []
test_hist = []
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 200 + 1):
    start = time.time()
    loss = train(model, train_loader, optimizer, epoch)
    L.append(loss)
    print("time {:.1f} sec:".format(time.time() - start))
    acc = test(model, test_loader)
    test_hist.append(acc)

Train Epoch: 1, Average Loss: 0.3593
time 1.3 sec:
Test set: Average loss: 0.3524, Accuracy: 1837/2069 (88.79%)

Train Epoch: 2, Average Loss: 0.3526
time 1.3 sec:
Test set: Average loss: 0.3545, Accuracy: 1837/2069 (88.79%)

Train Epoch: 3, Average Loss: 0.3516
time 1.3 sec:
Test set: Average loss: 0.3511, Accuracy: 1837/2069 (88.79%)

Train Epoch: 4, Average Loss: 0.3528
time 1.3 sec:
Test set: Average loss: 0.3535, Accuracy: 1837/2069 (88.79%)

Train Epoch: 5, Average Loss: 0.3502
time 1.3 sec:
Test set: Average loss: 0.3511, Accuracy: 1837/2069 (88.79%)

Train Epoch: 6, Average Loss: 0.3497
time 1.3 sec:
Test set: Average loss: 0.3564, Accuracy: 1837/2069 (88.79%)

Train Epoch: 7, Average Loss: 0.3518
time 1.3 sec:
Test set: Average loss: 0.3509, Accuracy: 1837/2069 (88.79%)

Train Epoch: 8, Average Loss: 0.3500
time 1.3 sec:
Test set: Average loss: 0.3509, Accuracy: 1837/2069 (88.79%)

Train Epoch: 9, Average Loss: 0.3538
time 1.3 sec:
Test set: Average loss: 0.3513, Accuracy: 183

Train Epoch: 73, Average Loss: 0.0505
time 1.3 sec:
Test set: Average loss: 0.0359, Accuracy: 2038/2069 (98.50%)

Train Epoch: 74, Average Loss: 0.0369
time 1.3 sec:
Test set: Average loss: 0.0784, Accuracy: 2020/2069 (97.63%)

Train Epoch: 75, Average Loss: 0.1032
time 1.3 sec:
Test set: Average loss: 0.0719, Accuracy: 2006/2069 (96.96%)

Train Epoch: 76, Average Loss: 0.0584
time 1.3 sec:
Test set: Average loss: 0.0351, Accuracy: 2041/2069 (98.65%)

Train Epoch: 77, Average Loss: 0.0513
time 1.3 sec:
Test set: Average loss: 0.0606, Accuracy: 2027/2069 (97.97%)

Train Epoch: 78, Average Loss: 0.0440
time 1.3 sec:
Test set: Average loss: 0.0179, Accuracy: 2059/2069 (99.52%)

Train Epoch: 79, Average Loss: 0.0362
time 1.3 sec:
Test set: Average loss: 0.0267, Accuracy: 2048/2069 (98.99%)

Train Epoch: 80, Average Loss: 0.0400
time 1.3 sec:
Test set: Average loss: 0.0183, Accuracy: 2060/2069 (99.57%)

Train Epoch: 81, Average Loss: 0.1877
time 1.3 sec:
Test set: Average loss: 0.0661, Accu

Train Epoch: 145, Average Loss: 0.0206
time 1.3 sec:
Test set: Average loss: 0.0520, Accuracy: 2041/2069 (98.65%)

Train Epoch: 146, Average Loss: 0.0211
time 1.3 sec:
Test set: Average loss: 0.0177, Accuracy: 2054/2069 (99.28%)

Train Epoch: 147, Average Loss: 0.0226
time 1.3 sec:
Test set: Average loss: 0.0125, Accuracy: 2057/2069 (99.42%)

Train Epoch: 148, Average Loss: 0.0189
time 1.3 sec:
Test set: Average loss: 0.0157, Accuracy: 2056/2069 (99.37%)

Train Epoch: 149, Average Loss: 0.0207
time 1.3 sec:
Test set: Average loss: 0.0211, Accuracy: 2053/2069 (99.23%)

Train Epoch: 150, Average Loss: 0.0238
time 1.3 sec:
Test set: Average loss: 0.0133, Accuracy: 2058/2069 (99.47%)

Train Epoch: 151, Average Loss: 0.0192
time 1.3 sec:
Test set: Average loss: 0.0169, Accuracy: 2055/2069 (99.32%)

Train Epoch: 152, Average Loss: 0.0680
time 1.3 sec:
Test set: Average loss: 0.0942, Accuracy: 1991/2069 (96.23%)

Train Epoch: 153, Average Loss: 0.1601
time 1.3 sec:
Test set: Average loss: 0.1

In [10]:
model.eval()
y_pred = []
out=[]
y_test = []
with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        output = model(data)
        output=F.softmax(output, dim=1)
        pred = output.max(1, keepdim=True)[1]  
        pred = pred.cpu().numpy().squeeze()
        y_pred.append(pred)
        out.append(output.cpu().numpy())
        y_test.append(target.numpy().squeeze())
y_pred=np.hstack(y_pred)
y_test=np.hstack(y_test)
out=np.vstack(out)

summary(y_pred,y_test)

confusion matrix:
 [[1830    4]
 [   7  228]]
Roc score: 0.9840
F1 score: 0.9764
Recall_score: 0.9702
Precision_score: 0.9828
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1834
           1       0.98      0.97      0.98       235

    accuracy                           0.99      2069
   macro avg       0.99      0.98      0.99      2069
weighted avg       0.99      0.99      0.99      2069



In [11]:
torch.save(model, './models/TFEQ.pth')

In [9]:
earth=np.where(y_pred==1)[0]
non_earth=np.where(y_pred==0)[0]

p_earth=out[earth,1].squeeze()

p_non_earth=out[non_earth,0].squeeze()

In [12]:
a = np.random.rand(100)
hist,bins = np.histogram(p_earth,bins=10,range=(0,1))
print(hist)
print(bins)


[  0   0   0   0   0   1   4   3   9 217]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [13]:
a = np.random.rand(100)
hist,bins = np.histogram(p_non_earth,bins=10,range=(0,1))
print(hist)
print(bins)

[   0    0    0    0    0    6    5    4    5 1816]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
